In [1]:
import os
import sys
sys.path.append('D:\\Users\\Hegxiten\\workspace\\Rutgers_Railway_security_research\\OOD_Train')

from datetime import datetime, timedelta
import numpy as np
import random
from train import Train
from infrastructure import Track, Block, BigBlock
from signaling import AutoSignal, HomeSignal, AutoPoint, ControlPoint

import networkx as nx


In [2]:
_node = {   0:ControlPoint(idx=0, ports=[0,1], MP=0.0), \
            1:AutoPoint(1), \
            2:AutoPoint(2), \
            3:ControlPoint(idx=3, ports=[0,1,3], ban_ports_by_port={1:[3],3:[1]}), \
            4:ControlPoint(idx=4, ports=[0,2,1], ban_ports_by_port={0:[2],2:[0]}), \
            5:AutoPoint(5), \
            6:ControlPoint(idx=6, ports=[0,1,3], ban_ports_by_port={1:[3],3:[1]}), \
            7:ControlPoint(idx=7, ports=[0,2,1], ban_ports_by_port={0:[2],2:[0]}), \
            8:AutoPoint(8), \
            9:AutoPoint(9), \
            10:ControlPoint(idx=10, ports=[0,1])}       
nbunch = [_node[i] for i in range(len(_node))]

_track = [  Track(nbunch[0], 1, nbunch[1], 0), Track(nbunch[1], 1, nbunch[2], 0), Track(nbunch[2], 1, nbunch[3], 0),\
            Track(nbunch[3], 1, nbunch[4], 0), Track(nbunch[3], 3, nbunch[4], 2, edge_key=1),\
            Track(nbunch[4], 1, nbunch[5], 0), Track(nbunch[5], 1, nbunch[6], 0),\
            Track(nbunch[6], 1, nbunch[7], 0), Track(nbunch[6], 3, nbunch[7], 2, edge_key=1),\
            Track(nbunch[7], 1, nbunch[8], 0), Track(nbunch[8], 1, nbunch[9], 0), Track(nbunch[9], 1, nbunch[10], 0)]
ebunch = [_track[i] for i in range(len(_track))]

# _node and _track will be parameters passed from outside in the future development
G = nx.MultiGraph()
for n in nbunch:
    G.add_node(n, attr=n.__dict__, instance=n)              
    # __dict__ of instances (CPs, ATs, Tracks) is pointing the same 
    # attribute dictionary as the node in the MultiGraph

for t in ebunch:
    G.add_edge(t.L_point, t.R_point, key=t.edge_key, attr=t.__dict__, instance=t)          
    # __dict__ of instances (CPs, ATs, Tracks) is pointing the same 
    # attribute dictionary as the edge in the MultiGraph
    # key is the index of parallel edges between two nodes
    t.L_point.track_by_port[t.L_point_port] = t.R_point.track_by_port[t.R_point_port] = t

for i in G.nodes():     # register the neighbor nodes as observers to each node
    i.neighbor_nodes.extend([n for n in G.neighbors(i)])              
    for n in G.neighbors(i):
        i.add_observer(n)

In [3]:
    F = G.copy()        
    # F is a shallow copy of G: attrbutes of G/F components 
    # are pointing at the same memory.

    def _get_new_edge(node, length=False):
        at_neighbor = [j for j in F.neighbors(i)]
        assert len(at_neighbor) == len(F.edges(i)) == 2
        edgetrk_L_points = [F[at_neighbor[0]][node][0]['instance'].L_point, F[node][at_neighbor[1]][0]['instance'].L_point]
        edgetrk_R_points = [F[at_neighbor[0]][node][0]['instance'].R_point, F[node][at_neighbor[1]][0]['instance'].R_point]
        edgetrk_L_points.remove(i)
        edgetrk_R_points.remove(i)
        new_edge_length = F[at_neighbor[0]][i][0]['instance'].length + F[i][at_neighbor[1]][0]['instance'].length
        if length:
            return edgetrk_L_points[0], edgetrk_R_points[0], new_edge_length
        else:  
            return edgetrk_L_points[0], edgetrk_R_points[0]

    for i in G.nodes():
        # only use G.nodes() instead of F.nodes() to get original nodes 
        # to avoid dictionary size changing issues. 
        # all the following graph updates are targeted on F
        if i.type == 'at':
            new_L_point, new_R_point, new_length = _get_new_edge(i, length=True)
            assert len(F[new_L_point][i]) == len(F[i][new_R_point]) == 1
            new_track =  Track( new_L_point, F[new_L_point][i][0]['instance'].L_point_port,\
                                new_R_point, F[i][new_R_point][0]['instance'].R_point_port,\
                                edge_key=0, length=new_length)

            F.remove_node(i)
            F.add_edge(new_L_point, new_R_point, attr=new_track.__dict__, instance=new_track)     
            # MultiGraph parallel edges are auto-keyed (0, 1, 2...)
            # default 0 as mainline, idx as track number

    for (u, v, k) in F.edges(keys=True):
        blk_path = nx.shortest_path(G, u, v)
        big_block_edges = [(blk_path[i], blk_path[i+1]) for i in range(len(blk_path) - 1)]
        big_block_instance = BigBlock(  u, F[u][v][k]['instance'].L_point_port,\
                                        v, F[u][v][k]['instance'].R_point_port,\
                                        edge_key=k, length=F[u][v][k]['instance'].length, \
                                        raw_graph=G, cp_graph=F)
        u.bigblock_by_port[F[u][v][k]['instance'].L_point_port] = v.bigblock_by_port[F[u][v][k]['instance'].R_point_port] = big_block_instance

        for (n, m) in big_block_edges:
            if G[n][m][k]['instance'] not in big_block_instance.tracks:
                big_block_instance.tracks.append(G[n][m][k]['instance'])
            # get the list of track unit components of a bigblock, and record in the instance

        F[u][v][k]['attr'] = big_block_instance.__dict__
        F[u][v][k]['instance'] = big_block_instance
        for t in F[u][v][k]['instance'].tracks:
            t.bigblock = F[u][v][k]['instance']

In [4]:
from system import System

In [5]:
sim_init_time = datetime.strptime('2018-01-10 10:00:00', "%Y-%m-%d %H:%M:%S")
sim_term_time = datetime.strptime('2018-01-10 15:30:00', "%Y-%m-%d %H:%M:%S")
sp_container = [random.uniform(0.01, 0.02) for i in range(20)]
acc_container = [random.uniform(2.78e-05*0.85, 2.78e-05*1.15) for i in range(20)]
dcc_container = [random.uniform(2.78e-05*0.85, 2.78e-05*1.15) for i in range(20)]
headway = 200 * random.random() + 400
sys = System(sim_init_time, sp_container, acc_container, dcc_container,
             dos_period=['2018-01-10 11:30:00', '2018-01-10 12:30:00'],  
             headway=headway, 
             tracks=[1,1,1,4,1,1,3,1,1,1], 
             dos_pos=-1)

In [9]:
new_train = Train(idx=sys.train_num, 
                  rank=sys.train_num, 
                  system=sys, 
                  init_time=sys.sys_time, 
                  init_direction=((None,None),(sys.control_points[0],0)), 
                  max_sp=sys.sp_container[sys.train_num % len(sys.sp_container)], 
                  max_acc=sys.acc_container[sys.train_num % len(sys.acc_container)], 
                  max_dcc=sys.dcc_container[sys.train_num % len(sys.dcc_container)])

In [10]:
new_train

AttributeError: 'NoneType' object has no attribute 'traffic_direction'

In [3]:
dir(ControlPoint)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'add_observer',
 'broadcast_route_to_autopoints',
 'cancel_bigblock_direction_by_route',
 'close_route',
 'current_invalid_routes',
 'current_route_by_port',
 'current_routes',
 'has_observer',
 'listener_updates',
 'mutex_routes_by_route',
 'observers',
 'open_route',
 'open_route_old',
 'remove_observer',
 'set_bigblock_direction_by_route',
 'update',
 'update_signal']

{'_observers': [AutoPoint2, ControlPoint4],
 'MP': 15.0,
 'idx': 3,
 'type': 'cp',
 'ports': [0, 1, 3],
 'available_ports_by_port': defaultdict(list, {0: [1, 3], 1: [0], 3: [0]}),
 'non_mutex_routes_by_route': defaultdict(list, {}),
 'ban_ports_by_port': {1: [3], 3: [1]},
 'all_valid_routes': [(0, 1), (1, 0), (0, 3), (3, 0)],
 '_current_routes': [],
 'neighbor_nodes': [AutoPoint2, ControlPoint4],
 'track_by_port': {0: Track MP: 10.0 to MP: 15.0 idx: 0,
  1: Track MP: 15.0 to MP: 20.0 idx: 0,
  3: Track MP: 15.0 to MP: 20.0 idx: 1},
 'signal_by_port': {0: HomeSignal of ControlPoint3, port: 0,
  1: HomeSignal of ControlPoint3, port: 1,
  3: HomeSignal of ControlPoint3, port: 3},
 'bigblock_by_port': {0: BigBlock MP: 0.0 to MP: 15.0 idx: 0,
  1: BigBlock MP: 15.0 to MP: 20.0 idx: 0,
  3: BigBlock MP: 15.0 to MP: 20.0 idx: 1}}

In [9]:
G[_node[3]][_node[4]][0]['instance'].train.append('k166')

In [11]:
_node[3].signal_by_port[0].sigpoint.track_by_port[1].is_Occupied

True

In [76]:
_node[0].__dict__

{'_observers': [AutoPoint1],
 'MP': 0.0,
 'idx': 0,
 'type': 'cp',
 'ports': [0, 1],
 'available_ports_by_port': defaultdict(list, {0: [1], 1: [0]}),
 'non_mutex_routes_by_route': defaultdict(list, {}),
 'ban_ports_by_port': defaultdict(list, {}),
 'all_valid_routes': [(0, 1), (1, 0)],
 '_current_routes': [(0, 1)],
 'neighbor_nodes': [AutoPoint1],
 'track_by_port': {1: Track MP: 0.0 to MP: 5.0 idx: 0},
 'signal_by_port': {0: HomeSignal of ControlPoint0, port: 0,
  1: HomeSignal of ControlPoint0, port: 1},
 'bigblock_by_port': {1: BigBlock MP: 0.0 to MP: 15.0 idx: 0}}

In [50]:
_node[3].signal_by_port[0].aspect

Aspect: r, route None

In [12]:
_node[2].signal_by_port[0]

AutoSignal of AutoPoint2, port: 0

In [13]:
_node[2].__dict__

{'_observers': [AutoPoint1, ControlPoint3],
 'MP': 10.0,
 'idx': 2,
 'type': 'at',
 'ports': [0, 1],
 'available_ports_by_port': {0: [1], 1: [0]},
 'non_mutex_routes_by_route': {},
 'ban_ports_by_port': {},
 'all_valid_routes': [(0, 1), (1, 0)],
 '_current_routes': [],
 'neighbor_nodes': [AutoPoint1, ControlPoint3],
 'track_by_port': {0: Track MP: 5.0 to MP: 10.0 idx: 0,
  1: Track MP: 10.0 to MP: 15.0 idx: 0},
 'signal_by_port': {0: AutoSignal of AutoPoint2, port: 0,
  1: AutoSignal of AutoPoint2, port: 1}}